### Plotly Figure Friday - 2024 week 51 - Worker Standing Hours

In [50]:
'''
This data set has inconsistent DATATYPES, measured as Hours, Pounds??, Days,
and Percentage. This script will only include Percentage, which comprises 
92% of the values.
┌────────────┬──────┬───────┐
│ DATATYPE   ┆ PCT  ┆ count │
│ ---        ┆ ---  ┆ ---   │
│ str        ┆ f64  ┆ u32   │
╞════════════╪══════╪═══════╡
│ Percentage ┆ 92.5 ┆ 45986 │
│ Hours      ┆ 3.7  ┆ 1833  │
│ Pounds     ┆ 1.6  ┆ 789   │
│ Days       ┆ 2.2  ┆ 1089  │
└────────────┴──────┴───────┘

Percentages starting with less than symbols, will have the < symbols stripped, 
and the values cut in half. For example, <1 will be represented by 0.5.

Percentages starting with greater than symbols, will have the > symbols stripped, 
and the values averaged with 100. For example, >99 will be represented by 99.5
'''

import plotly.express as px
import polars as pl
#-------------------------------------------------------------------------------
#    Read csv data to polars dataframe.  ~45% of the data have percentage values 
#    starting with < or >. These symbols are removed. Values starting with are 
#    cut in half, values starting with < are averaged with 100.
#-------------------------------------------------------------------------------
df_source = list_col_names = (
    pl.read_csv(
        'ors-limited-dataset.csv',
    )
    .filter(pl.col('DATATYPE') == 'Percentage')
    .rename({'ESTIMATE': 'ORIGINAL_ESTIMATE'})
    .with_columns(
        ESTIMATE = pl.when(  # if value starts with less than or greater than
                        pl.col('ORIGINAL_ESTIMATE').str.starts_with('<') 
                        |
                        pl.col('ORIGINAL_ESTIMATE').str.starts_with('>')
                    )
                    .then(pl.col('ORIGINAL_ESTIMATE').str.slice(1))
                    .otherwise('ORIGINAL_ESTIMATE')
    )
    .with_columns((pl.col('ESTIMATE').cast(pl.Float32).round(1)))
    # for percentages starting with less than, remove <, and use 1/2 the value
    .with_columns(
        ESTIMATE = pl.when(pl.col('ORIGINAL_ESTIMATE').str.starts_with('<'))
                    .then(pl.col('ESTIMATE')/2.0)
                    .otherwise('ESTIMATE')
    )
    # for percentages starting with greater than, remove >, and average with 100
    .with_columns(
        ESTIMATE = pl.when(pl.col('ORIGINAL_ESTIMATE').str.starts_with('>'))
                    .then((100 + pl.col('ESTIMATE'))/2.0)
                    .otherwise('ESTIMATE')
    )
    # scale percentages to range from 0 to 1 for proper formatting as PCTs
    .with_columns((pl.col('ESTIMATE')/100.0).round(3))
)
df_source.sample(10) # .filter(pl.col('ORIGINAL').str.starts_with('>'))

OCCUPATION,ESTIMATE TEXT,CATEGORY,DATATYPE,ORIGINAL_ESTIMATE,ESTIMATE
str,str,str,str,str,f32
"""Loan officers""","""Percent of workers that were e…","""Heights""","""Percentage""","""<0.5""",0.002
"""Paralegals and legal assistant…","""Percent of workers that were e…","""Extreme cold""","""Percentage""","""<0.5""",0.002
"""Correctional officers and jail…","""Percent of workers that requir…","""Reaching at or below the shoul…","""Percentage""","""86.5""",0.865
"""Computer and information syste…","""Percent of workers that did no…","""Work review: Presence of super…","""Percentage""","""61.1""",0.611
"""Customer service representativ…","""Percent of workers that were n…","""Humidity""","""Percentage""",""">99.5""",0.998
"""Human resources assistants, ex…","""Percent of workers that did no…","""Climbing work-related ramps or…","""Percentage""",""">90""",0.95
"""Receptionists and information …","""Percent of workers that could …","""Low postures""","""Percentage""","""21.3""",0.213
"""Licensed practical and license…","""Percent of workers that requir…","""Gross manipulation""","""Percentage""","""<0.5""",0.002
"""Switchboard operators, includi…","""Percent of workers that were i…","""Proximity to moving mechanical…","""Percentage""","""<0.5""",0.002


In [28]:
df_source.filter(pl.col('ORIGINAL').str.starts_with('>'))

OCCUPATION,ESTIMATE TEXT,CATEGORY,DATATYPE,ORIGINAL,ESTIMATE
str,str,str,str,str,f32
"""All workers""","""Percent of workers that requir…","""Fine manipulation""","""Percentage""",""">99.5""",50.497501
"""All workers""","""Percent of workers that requir…","""Gross manipulation""","""Percentage""",""">99.5""",50.497501
"""All workers""","""Percent of workers that requir…","""Hearing requirements""","""Percentage""",""">99.5""",50.497501
"""Management occupations""","""Percent of workers that requir…","""Personal contacts: People skil…","""Percentage""",""">99.5""",50.497501
"""Management occupations""","""Percent of workers that were n…","""Heavy vibrations""","""Percentage""",""">99.5""",50.497501
…,…,…,…,…,…
"""Stockers and order fillers""","""Percent of workers that requir…","""Gross manipulation""","""Percentage""",""">99.5""",50.497501
"""Stockers and order fillers""","""Percent of workers that requir…","""Hearing requirements""","""Percentage""",""">99.5""",50.497501
"""Stockers and order fillers""","""Percent of workers that were n…","""Low postures""","""Percentage""",""">99.5""",50.497501


In [ ]:
print(1/0)

In [48]:
import plotly.express as px
import polars as pl

#-------------------------------------------------------------------------------
#    Read excel speadsheet to polars dataframe, df_source.
#-------------------------------------------------------------------------------
df_source = list_col_names = (
    pl.read_excel(
        'CMO-Historical-Data-Monthly.xlsx',
        sheet_name='Monthly Prices',
        has_header=True,
        read_options={'header_row': 4}
    )
    .rename({'__UNNAMED__0': 'MONTH'})
)
#-------------------------------------------------------------------------------
#    Make a list of useful columns names using rows 7 & 6
#-------------------------------------------------------------------------------
df_header_info = (
    df_source   
    .head(2)
    .transpose(include_header=True)
    .rename({
        'column'         : 'DESC',
        'column_0'       : 'UNITS',
        'column_1'       : 'ITEM',
        })
    .with_columns(
         column_names = # if units column is blank, just take name without units
             pl.when(pl.col('ITEM').is_not_null())
               .then(pl.col('ITEM') + pl.lit(' ') + pl.col('UNITS'))
               .otherwise(pl.col('DESC'))
    )
)
list_column_names = (
    df_header_info
    .select(pl.col('column_names'))
    .to_series().to_list()
)

#-------------------------------------------------------------------------------
#    Make a working dataframe, convert data columns from  strings to floats
#-------------------------------------------------------------------------------
                                                                               
data_col_names = df_source.columns[1:]

df = (
    df_source
    .with_row_index()
    .filter(pl.col('index') > 1)
    .drop('index')
    .with_columns(pl.col(data_col_names).cast(pl.Float64, strict=False))
    .with_columns(YEAR_STR = pl.col('MONTH').str.slice(0,4))
    .with_columns(MONTH_STR = pl.col('MONTH').str.slice(-2,2))
    .with_columns(DATE_STR = (pl.col('YEAR_STR') + '-' + pl.col('MONTH_STR')))
    .with_columns(DATE = pl.col('DATE_STR').str.to_date('%Y-%m'))
    .drop(['YEAR_STR', 'MONTH_STR', 'DATE_STR'])  # temporary cols to make DATE

)
# rename all columns, and add newly added DATE column to this list
df.columns=list_column_names + ['DATE']
# reorder the columns, with DATE at the far left, followed by data columns
df = (
    df
    .select(['DATE'] + list_column_names)
    .drop('MONTH')
)

#-------------------------------------------------------------------------------
#    use px line to plot commodity prices - Coffee or Tea?
#-------------------------------------------------------------------------------
df_plotting = (   # add average coffee price
    df
    .with_columns(
        (   # calculate average coffee prices, AVG_TEA included in dataset
            (pl.col('COFFEE_ARABIC ($/kg)') + 
             pl.col('COFFEE_ROBUS ($/kg)'))
            /2.0).alias('COFFEE_AVG ($/kg)')
        )
)
fig = px.line(
    df_plotting,
    'DATE',
    ['TEA_AVG ($/kg)', 'COFFEE_AVG ($/kg)'],
    template='simple_white',
    height=400, width=800
)

#-------------------------------------------------------------------------------
#    Touch up and annotate
#-------------------------------------------------------------------------------
tea_color = '#006400'
coffee_color = '#D2691E'
fig['data'][0]['line']['color']=tea_color
fig['data'][1]['line']['color']=coffee_color
date_newest = df_plotting['DATE'].to_list()[-1]
coffee_newest = df_plotting['COFFEE_AVG ($/kg)'].to_list()[-1]
tea_newest = df_plotting['TEA_AVG ($/kg)'].to_list()[-1]
offset = 10
fig.add_annotation(
    x=date_newest, xanchor='left',
    y=coffee_newest, 
    showarrow=False,
    text='<b>COFFEE</b>',
    align='right',
    font=dict(size=14, color=coffee_color),
)
fig.add_annotation(
    x=date_newest, xanchor='left',
    y=tea_newest, 
    showarrow=False,
    text='<b>TEA</b>',
    font=dict(size=14, color=tea_color),
)
fig.update_layout(
    showlegend=False,
    title_text = (
        'COFFEE OR TEA?<br>' +
        '<sup>Coffee jitter is not just biological</sup>'
        ),
    xaxis_title='', yaxis_title='AVERAGE PRICE ($/kg)'
    )
fig.write_html('Plotly_FF_2024_50_Coffee_Tea.html')
fig.show()

FileNotFoundError: CMO-Historical-Data-Monthly.xlsx